In [2]:
import pandas as pd
import numpy as np

# Configuração da semente para reprodutibilidade
np.random.seed(42)
n_rows = 1_000_000

# 1. Definição de blocos de construção de frases para cada sentimento
# Isso garante que o texto faça sentido gramatical e corresponda ao label.

# Reviews Positivos
pos_starts = ["Adorei", "Gostei muito do", "Excelente", "Recomendo o", "Muito bom este"]
pos_adjs = ["produto", "serviço", "atendimento", "item", "material"]
pos_ends = ["de ótima qualidade.", "chegou rápido!", "superou expectativas.", "muito eficiente.", "vale a pena."]

# Reviews Negativos
neg_starts = ["Odiei", "Não gostei do", "Péssimo", "Não recomendo o", "Horrível este"]
neg_adjs = ["produto", "acabamento", "suporte", "material", "aparelho"]
neg_ends = ["veio com defeito.", "demorou muito.", "quebrou logo.", "dinheiro jogado fora.", "de baixa qualidade."]

# Reviews Neutros
neu_starts = ["O produto é ok", "Chegou no prazo", "Normal", "Razoável", "Nada de mais"]
neu_adjs = [", mas esperava mais do", ", contudo o", ", porém o", ", o"]
neu_ends = ["acabamento é simples.", "preço é justo.", "cumpre o básico.", "não impressionou.", "atende, mas tem falhas."]

# 2. Função para gerar texto sintético rápido (CORRIGIDA)
def generate_text(sentiment_list, n):
    starts = np.random.choice(sentiment_list['starts'], n)
    adjs = np.random.choice(sentiment_list['adjs'], n)
    ends = np.random.choice(sentiment_list['ends'], n)

    # CORREÇÃO: Usar np.char.add em vez de np.core.defchararray.add
    # Concatena: "Inicio " + "Adjetivo" + " " + "Final"

    # Passo 1: Inicio + " "
    part1 = np.char.add(starts, " ")

    # Passo 2: " " + Final
    part2 = np.char.add(" ", ends)

    # Passo 3: Adjetivo + Passo 2
    part3 = np.char.add(adjs, part2)

    # Final: Passo 1 + Passo 3
    return np.char.add(part1, part3)

# 3. Gerar os dados
# Vamos distribuir: 40% Positivo, 40% Negativo, 20% Neutro
n_pos = int(n_rows * 0.4)
n_neg = int(n_rows * 0.4)
n_neu = n_rows - (n_pos + n_neg)

print("Gerando textos...")

# Gerar textos positivos
texts_pos = generate_text({'starts': pos_starts, 'adjs': pos_adjs, 'ends': pos_ends}, n_pos)
labels_pos = np.full(n_pos, 'Positivo')

# Gerar textos negativos
texts_neg = generate_text({'starts': neg_starts, 'adjs': neg_adjs, 'ends': neg_ends}, n_neg)
labels_neg = np.full(n_neg, 'Negativo')

# Gerar textos neutros
texts_neu = generate_text({'starts': neu_starts, 'adjs': neu_adjs, 'ends': neu_ends}, n_neu)
labels_neu = np.full(n_neu, 'Neutro')

# 4. Concatenar e Embaralhar
all_texts = np.concatenate([texts_pos, texts_neg, texts_neu])
all_labels = np.concatenate([labels_pos, labels_neg, labels_neu])

# Criar DataFrame
df = pd.DataFrame({
    'review_id': range(1, n_rows + 1),
    'texto': all_texts,
    'sentimento': all_labels
})

# Embaralhar as linhas para não ficarem ordenadas por sentimento
df = df.sample(frac=1).reset_index(drop=True)

# 5. Visualizar resultado
print(f"Dataframe gerado com {len(df)} linhas.")
print(df['sentimento'].value_counts())
print("\nPrimeiras 5 linhas:")
print(df.head())

# Opcional: Salvar em CSV (Cuidado: arquivo terá cerca de 50-60MB)
# df.to_csv('reviews_1m_pt.csv', index=False)

Gerando textos...
Dataframe gerado com 1000000 linhas.
sentimento
Positivo    400000
Negativo    400000
Neutro      200000
Name: count, dtype: int64

Primeiras 5 linhas:
   review_id                                          texto sentimento
0     991443       O produto é ok , o acabamento é simples.     Neutro
1     274049     Gostei muito do item superou expectativas.   Positivo
2     538279  Não recomendo o produto dinheiro jogado fora.   Negativo
3     790953       Não gostei do aparelho veio com defeito.   Negativo
4     613093          Não recomendo o produto quebrou logo.   Negativo


In [3]:
df.head

<bound method NDFrame.head of         review_id                                          texto sentimento
0          991443       O produto é ok , o acabamento é simples.     Neutro
1          274049     Gostei muito do item superou expectativas.   Positivo
2          538279  Não recomendo o produto dinheiro jogado fora.   Negativo
3          790953       Não gostei do aparelho veio com defeito.   Negativo
4          613093          Não recomendo o produto quebrou logo.   Negativo
...           ...                                            ...        ...
999995      54117             Excelente produto muito eficiente.   Positivo
999996     721450                Odiei acabamento demorou muito.   Negativo
999997     868403                    Normal , o cumpre o básico.     Neutro
999998     122526           Excelente atendimento chegou rápido!   Positivo
999999     218997         Excelente atendimento muito eficiente.   Positivo

[1000000 rows x 3 columns]>